# Installation de selenium

Décommetez et exécutez le code ci-dessous

In [1]:
!pip install selenium

In [2]:
import selenium

In [3]:
from selenium import  webdriver

In [4]:
driver_path = "C:/Users/NKA/Downloads/chromedriver.exe"

In [7]:
# Créer un navigateur
# téléchargerr le driver
# * Firefox : https://github.com/mozilla/geckodriver/releases
# Chrome : https://chromedriver.chromium.org/downloads
driver = webdriver.Chrome(executable_path= driver_path)

In [8]:
driver.get("https://www.google.com/flights?hl=fr&gl=fr")

In [57]:
# page = driver.page_source
# page[:1000]

In [74]:
import time

In [81]:
time.sleep(2)
print("aaaaaaa")

aaaaaaa


In [80]:
print("aaaaaaa")

aaaaaaa


In [113]:
driver.get("https://www.google.com/flights?hl=fr&gl=fr")

In [122]:
def prix_vol(depart = "Paris", arrivee = "Abidjan"):
    driver.get("https://www.google.com/flights?hl=fr&gl=fr")
    time.sleep(1)
    remplir_depart(depart)
    remplir_arrivee(arrivee)
    rechercher()
    time.sleep(3)
    derouler()

In [123]:
prix_vol()

In [41]:
origin_airport = driver.find_element_by_css_selector(
    "div[data-flt-ve='origin_airport'][class*='flt-input']")

In [42]:
# len(origin_airport)

In [43]:
origin_airport.get_attribute("outerHTML")

'<div data-flt-ve="origin_airport" jstcache="6990" role="presentation" jsaction="jsl._;f_f:jsl._;ica_bc:jsl._;keydown:jsl._" class="flt-input gws-flights-form__input-container gws-flights__flex-box gws-flights-form__airport-input gws-flights-form__swapper-right" jsan="0.data-flt-ve,22.jsaction,t-ch-jmKqQoDY,7.flt-input,7.gws-flights-form__input-container,7.gws-flights__flex-box,7.gws-flights-form__airport-input,7.gws-flights-form__swapper-right,0.role,21.jssc"><div jstcache="6723" jscontroller="pSNSIe" jsaction="rcuQ6b:npT2md" class="gws-flights-widgets-inputcapturearea__capture-area-wrapper"><span jsname="bOjMyf" aria-live="polite" jstcache="6657" style="display:none" jsvs="&quot;bOjMyf&quot;;" jsan="0.jsname,5.display,0.aria-live,t-uU8ghUBFeaU" id="flt-ica-5"> <span jstcache="6756">D\'où partez-vous&nbsp;?</span> <jsl jstcache="6757"><span><span>Abidjan</span><span class="gws-flights-form__iata-code">ABJ</span></span></jsl> </span><textarea jsname="LlUqIb" role="textbox" tabindex="0"

In [44]:
origin_airport.click() # Pour cliquer et faire apparaître le formulaire 

In [38]:
# origin_airport.send_keys("abidjan")

In [45]:
origin_airport2 = driver.find_element_by_css_selector(
    "input[placeholder*='partez-vous']")

In [46]:
origin_airport2.clear()

In [47]:
origin_airport2.send_keys("paris")

In [48]:
origin_airport3 = driver.find_element_by_css_selector(
    ".fsapp-option-content .fsapp-option-city-name")

In [49]:
origin_airport3.click()

In [50]:
destination_airport = driver.find_element_by_css_selector(
    "div[data-flt-ve='destination_airport'][class*='flt-input']")

In [51]:
destination_airport.click()

In [52]:
destination_airport2 = driver.find_element_by_css_selector(
    "input[placeholder*='allez-vous']")

In [53]:
destination_airport2.send_keys("New York")

In [54]:
destination_airport3 = driver.find_element_by_css_selector(
    ".fsapp-option-content .fsapp-option-city-name")

In [55]:
destination_airport3.click()

In [ ]:
"gws-flights-form__search-button-wrapper"

In [25]:
search_button = driver.find_element_by_css_selector(
".gws-flights-form__search-button-wrapper floating-action-button.gws-flights-form__search-button")

In [26]:
search_button.click()

In [31]:
expand_button = driver.find_element_by_css_selector(
                        ".gws-flights-results__dominated-toggle")

In [32]:
expand_button.click()

In [124]:
import requests

In [126]:
chat = requests.get(
"https://lemagduchat.ouest-france.fr/images/dossiers/2018-11/chat-drole-113730.jpg")

In [127]:
chat_text = chat.text

In [129]:
# chat_text[:1000]

In [130]:
with open("chat.jpg", "wb") as f:
    f.write(chat.content)

In [105]:
def remplir_depart(depart):
    css_origin_airport = "div[data-flt-ve='origin_airport'][class*='flt-input']"
    origin_airport = driver.find_element_by_css_selector(css_origin_airport)
    
    
    origin_airport.click() # Pour cliquer et faire apparaître le formulaire 
    time.sleep(1)
    
    css_origin_airport2 = "input[placeholder*='partez-vous']"
    origin_airport2 = driver.find_element_by_css_selector(
                                            css_origin_airport2)
    origin_airport2.clear()
    origin_airport2.send_keys(depart)
    time.sleep(3)

    css_origin_airport3 = ".fsapp-option-content .fsapp-option-city-name"
    origin_airport3 = driver.find_element_by_css_selector(
                                        css_origin_airport3)
    origin_airport3.click()

In [106]:
def remplir_arrivee(arrivee):
    css_destination_airport = "div[data-flt-ve='destination_airport'][class*='flt-input']"
    destination_airport = driver.find_element_by_css_selector(
        css_destination_airport)
    
    
    destination_airport.click() # Pour cliquer et faire apparaître le formulaire 
    time.sleep(1)
    
    css_destination_airport2 = "input[placeholder*='allez-vous']"
    destination_airport2 = driver.find_element_by_css_selector(
                                            css_destination_airport2)
    destination_airport2.clear()
    destination_airport2.send_keys(arrivee)
    time.sleep(3)

    css_destination_airport3 = ".fsapp-option-content .fsapp-option-city-name"
    destination_airport3 = driver.find_element_by_css_selector(
                                        css_destination_airport3)
    destination_airport3.click()

In [112]:
def rechercher():
    search_button = driver.find_element_by_css_selector(
            ".gws-flights-form__search-button-wrapper " 
        + "floating-action-button.gws-flights-form__search-button")
    search_button.click()
    
def derouler():
    expand_button = driver.find_element_by_css_selector(
                        ".gws-flights-results__dominated-toggle")
    expand_button.click()